# Семинар 6. 
# Расчёт длительности А/B теста + продвинутые топики
## Домашнее задание.

In [2]:
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd
from typing import Union
import plotly.graph_objects as go
from scipy import stats
from math import asin
import numpy as np

### Задание 1. 
Продакт на главной mail.ru решил протестировать в рекомендательной ленте контента вместо карточек со статьями видеоплеер с короткими видео. Нынешний таймспент на юзера в день в среднем равен 25 минут, а стандартная ошибка (SD) равна 156.    
Мы предполагаем, что в новой версии таймспент на юзера в день изменится на 10%. Средний трафик 20000 человек в день.   
Посчитайте сколько дней необходимо держать эксперимент при alpha = 5% и beta = 20% 

Исходные данные:

In [9]:
user_time_1 = 25   # средний таймспент юзера
std_1, std_2 = 156, 156 * 1.15          # стандартная ошибка. Принимаем на 15% больше для тестовой группы
delta = 0.1        # изменение таймспента юзера
user_time_2 = np.round(user_time_1 + user_time_1 * delta , 2)  # новый таймспент юзера
DAU = 20000   # количество посетителей сайта в день
alfa = 0.05
beta = 0.2
user_time_2

27.5

По имеющимся данным рассчитаем размер выборки для оценки изменений:

1. Найдем effectsize для непрерывной метрики

In [10]:
def calc_continuous_es_alt(mean_control: Union[float, int],
                            std_control: Union[float, int],
                            mean_test: Union[float, int],
                            std_test: Union[float, int]):
    
    effect_size = (mean_test - mean_control) / ((std_control**2 + std_test**2) / 2) ** 0.5
    return effect_size

In [11]:
es_count = calc_continuous_es_alt(user_time_1, std_1, user_time_2, std_2)
es_count

0.014871423685692022

2. Рассчитаем минимально необходимую выборку для полученного effectsize 

In [12]:
def calc_sample_size_continous(effect_size: float, 
                              alpha: float = 0.05,
                              beta: float = 0.2,
                              ratio: Union[float, int] = 1):
    
    n = tt_ind_solve_power(effect_size=effect_size, 
                              alpha=alpha,
                              power=1-beta,
                              ratio=ratio,
                          )
    return int(n * 2)

In [13]:
print(f'Минимальное количество пользователей сайта, необходимое для проверки результатов эксперимента:{calc_sample_size_continous(es_count)}')

Минимальное количество пользователей сайта, необходимое для проверки результатов эксперимента:141960


3. Расситаем длительность эксперимента в днях.    

Для этого полученное значение минимально необходимой выборки разделим на количество пользователей, посещающих сайт в день:

In [14]:
print(f'Количество дней эксперимента: {calc_sample_size_continous(es_count) / DAU}')

Количество дней эксперимента: 7.098


**Ответ:    
Для оценки результатов изменений эксперимент должен длиться 7 дней при alpha = 5% и beta = 20%.****

### Задание 2. 
Наша продуктовая команда в ecommerce магазине планирует запустить тест, направленный на ускорение загрузки сайта.    
Одна из основных метрик bounce rate в GA = 40%.    
Мы предполагаем, что при оптимизации сайта она изменится минимум на 20%.   
Средний трафик  4000 человек в день.    
Посчитайте сколько нам нужно дней держать эксперимент при alpha = 5% и beta = 20%

In [27]:
br_1 = 0.4   # текущий bounce rate
br_2 = br_1 - (br_1 * 0.2) # ожидаемый bounce rate
DAU = 4000  # количество посетителей сайта в день

По имеющимся данным рассчитаем размер выборки для оценки изменений:

1. Найдем effectsize для дискретной метрики

In [28]:
def calc_proportion_es(prob1: float, prob2: float):
    
    return abs(proportion_effectsize(prob1, prob2))

In [29]:
es_prop = calc_proportion_es(br_1, br_2)
es_prop

0.16690997264630925

2. Рассчитаем минимально необходимую выборку для полученного effectsize 

In [30]:
def calc_sample_size_proportion(effect_size: float, 
                              alpha: float = 0.05,
                              beta: float = 0.2,
                              ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size, 
                              alpha=alpha,
                              power=1-beta,
                              ratio=ratio,
                          )
    return int(n * 2)

In [31]:
print(f'Минимальное количество пользователей сайта, необходимое для проверки результатов эксперимента:{calc_sample_size_proportion(es_prop)}')

Минимальное количество пользователей сайта, необходимое для проверки результатов эксперимента:1126


3. Расситаем длительность эксперимента в днях.    

Для этого полученное значение минимально необходимой выборки разделим на количество пользователей, посещающих сайт в день:

In [32]:
print(f'Количество дней эксперимента: {calc_sample_size_proportion(es_prop) / DAU}')

Количество дней эксперимента: 0.2815


**Ответ:    
Для оценки результатов изменений эксперимент должен длиться 0,3 дня при alpha = 5% и beta = 20%.**